**Table of contents**<a id='toc0_'></a>    
- [Forward Kinematics Using Trigonometric Functions](#toc1_)    
- [Forward Kinematics Using Trigonometric Functions for a "2-Axis Rotational Joint Robot Arm"](#toc2_)    
    - [Starting pybullet](#toc2_1_1_)    
    - [Initial Setup for pybullet](#toc2_1_2_)    
    - [Generating the Robot Arm](#toc2_1_3_)    
    - [Defining the Function for Forward Kinematics Using Trigonometric Functions](#toc2_1_4_)    
    - [Executing Forward Kinematics Using Trigonometric Functions](#toc2_1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Forward Kinematics Using Trigonometric Functions](#toc0_)

In this notebook, we will explain the procedure to implement "forward kinematics using trigonometric functions" with a 2-axis robot arm in Pybullet.

(For a manual summarizing the functions available in pybullet, refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)


# <a id='toc2_'></a>[Forward Kinematics Using Trigonometric Functions for a "2-Axis Rotational Joint Robot Arm"](#toc0_)

From here, we will explain the theory and implementation of forward kinematics using trigonometric functions for a "2-axis rotational joint robot arm."

### <a id='toc2_1_1_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


### <a id='toc2_1_2_'></a>[Initial Setup for pybullet](#toc0_)

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add paths to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the elapsed time per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position and other parameters in GUI mode
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


### <a id='toc2_1_3_'></a>[Generating the Robot Arm](#toc0_)
This time, we will generate a 2-axis robot arm `simple_2d_arm.urdf`.  
The robot is structured as shown in the diagram below (sensors are explained in `robot_arm_sensor.ipynb`). 

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# Load the robot
arm_start_pos = [0, 0, 0.1]  # Set the initial position (x, y, z)
arm_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True) # Fix the root link with useFixedBase=True to prevent the robot from falling

# Set the camera position and other parameters in GUI mode
camera_distance = 1.5
camera_yaw = 180.0 # deg
camera_pitch = -10 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


### <a id='toc2_1_4_'></a>[Defining the Function for Forward Kinematics Using Trigonometric Functions](#toc0_)
We will define the function for "forward kinematics using trigonometric functions."


In [4]:
import numpy as np

def forward_kinematics2d(link_length, theta):
    """
    Calculate forward kinematics (using trigonometric functions) in a 2D plane
    Parameters
    ----------
    link_length : float
        Length of the link
    theta : float
        Rotation angle (rad)

    Returns
    -------
    x: x-coordinate of the link's end
    y: y-coordinate of the link's end
    """
    x = link_length * np.cos(theta)
    y = link_length * np.sin(theta)
    return x, y

### <a id='toc2_1_5_'></a>[Executing Forward Kinematics Using Trigonometric Functions](#toc0_)

Next, by running the code below, forward kinematics will be calculated using the angles set in `link1_angle_deg` and `link2_angle_deg`, and the calculated "end-effector position" will be displayed on the screen. (Note that the origin is based on the root of the robot arm (i.e., the link1 coordinate system).)

Additionally, the angles of each joint will be set to `link1_angle_deg` and `link2_angle_deg`, so please verify that the result of the forward kinematics matches the actual position of the end-effector.

In [5]:
import math

LINK1_LENGTH = 0.5   # Length of link1 (length of link1 in "simple2d_arm.urdf")
LINK2_LENGTH = 0.55  # Length of link2 (length of link2 + force_sensor_link in "simple2d_arm.urdf")

# Change these values to see different results ##########
link1_angle_deg = 30
link2_angle_deg = 90
###############################################

link1_angle_rad = math.radians(link1_angle_deg)
link2_angle_rad = math.radians(link2_angle_deg)

# Calculate the initial end-effector position P_current using forward kinematics with trigonometric functions
# Position of the end of link1 when viewed from the root of link1
x1_to_2, y1_to_2 = forward_kinematics2d(LINK1_LENGTH, link1_angle_rad)

# Position of the end of link2 (i.e., the end-effector position) when viewed from the root of link2
x2_to_e, y2_to_e = forward_kinematics2d(LINK2_LENGTH, link1_angle_rad + link2_angle_rad)

# Position of the end-effector when viewed from the root of link1 (origin coordinates)
xe = x1_to_2 + x2_to_e
ye = y1_to_2 + y2_to_e

# Display the result in the terminal
print(f"x, y = ({xe}, {ye})")

# Display the result on the screen
text_position = [0.5, 0.0, 2.0]
life_time = 10.0 # Display duration (seconds)
pybullet.addUserDebugText(f"x, y = ({xe:.2f}, {ye:.2f})", text_position, textSize=2, lifeTime=life_time)

# Move the joints to verify that the result of the forward kinematics matches the actual end-effector position
pybullet.resetJointState(arm_id, 0, link1_angle_rad)
pybullet.resetJointState(arm_id, 1, link2_angle_rad)
pybullet.stepSimulation()

x, y = (0.15801270189221944, 0.7263139720814413)


()

: 